# telecommunications: Iceberg and Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Iceberg and Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a telecommunications analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Iceberg?

Apache Iceberg is an open table format for huge analytic datasets that provides:

- **Schema evolution**: Add, drop, rename, update columns without rewriting data
- **Partition evolution**: Change partitioning without disrupting queries
- **Time travel**: Query historical data snapshots for auditing and rollback
- **ACID transactions**: Reliable concurrent read/write operations
- **Cross-engine compatibility**: Works with Spark, Flink, Presto, Hive, and more
- **Open ecosystem**: Apache 2.0 licensed, community-driven development

### Delta Universal Format with Iceberg

Delta Universal Format enables Iceberg compatibility while maintaining Delta's advanced features like liquid clustering. This combination provides:

- **Best of both worlds**: Delta's performance optimizations with Iceberg's openness
- **Multi-engine access**: Query the same data from different analytics engines
- **Future-proof architecture**: Standards-based approach for long-term data investments
- **Enhanced governance**: Rich metadata and catalog integration

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Network Performance Monitoring and Customer Experience Analytics

We'll analyze telecommunications network performance and customer usage data. Our clustering strategy will optimize for:

- **Customer-specific queries**: Fast lookups by subscriber ID
- **Time-based analysis**: Efficient filtering by call/service date
- **Network performance patterns**: Quick aggregation by cell tower and service quality metrics

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [1]:
# Create telecommunications catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS telecom")

spark.sql("CREATE SCHEMA IF NOT EXISTS telecom.analytics")

print("Telecommunications catalog and analytics schema created successfully!")

Telecommunications catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `network_usage_uf` table will store:

- **subscriber_id**: Unique customer identifier
- **usage_date**: Date and time of service usage
- **service_type**: Type (Voice, Data, SMS, Streaming)
- **data_volume**: Data consumed (GB)
- **call_duration**: Call length (minutes)
- **cell_tower_id**: Network cell tower identifier
- **signal_quality**: Network signal strength (0-100)

### Clustering Strategy

We'll cluster by `subscriber_id` and `usage_date` because:

- **subscriber_id**: Customers generate multiple service interactions, grouping their usage patterns together
- **usage_date**: Time-based queries are critical for billing cycles, network planning, and customer behavior analysis
- This combination optimizes for both customer analytics and temporal network performance monitoring

In [1]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
data_schema = StructType([
    StructField("subscriber_id", StringType(), True),
    StructField("usage_date", TimestampType(), True),
    StructField("service_type", StringType(), True),
    StructField("data_volume", DoubleType(), True),
    StructField("call_duration", DoubleType(), True),
    StructField("cell_tower_id", StringType(), True),
    StructField("signal_quality", IntegerType(), True)])

spark.sql("""

CREATE TABLE IF NOT EXISTS telecom.analytics.network_usage_uf (
    subscriber_id STRING,
    usage_date TIMESTAMP,
    service_type STRING,
    data_volume DECIMAL(10,3),
    call_duration DECIMAL(8,2),
    cell_tower_id STRING,
    signal_quality INT

)

USING DELTA

TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (subscriber_id, usage_date)

""")

print("Delta table with Iceberg compatibility and liquid clustering created successfully!")

print("Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.")

Delta table with Iceberg compatibility and liquid clustering created successfully!
Universal format enables Iceberg features while CLUSTER BY (columns) optimizes data layout.


## Step 3: Generate Telecommunications Sample Data

### Data Generation Strategy

We'll create realistic telecommunications usage data including:

- **10,000 subscribers** with multiple service interactions over time
- **Service types**: Voice calls, Data usage, SMS, Video streaming
- **Realistic usage patterns**: Peak hours, weekend vs weekday patterns, roaming
- **Network infrastructure**: Multiple cell towers with varying signal quality

### Why This Data Pattern?

This data simulates real telecommunications scenarios where:

- Customer usage varies by time of day and service type
- Network performance impacts customer experience
- Billing and service quality require temporal analysis
- Capacity planning depends on usage patterns
- Fraud detection needs real-time monitoring

In [1]:
# Generate sample telecommunications usage data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define telecommunications data constants

SERVICE_TYPES = ['Voice', 'Data', 'SMS', 'Streaming']

CELL_TOWERS = ['TOWER_NYC_001', 'TOWER_LAX_002', 'TOWER_CHI_003', 'TOWER_HOU_004', 'TOWER_MIA_005', 'TOWER_SFO_006', 'TOWER_SEA_007']

# Base usage parameters by service type

USAGE_PARAMS = {

    'Voice': {'avg_duration': 5.0, 'frequency': 8, 'data_volume': 0.0},

    'Data': {'avg_duration': 0.0, 'frequency': 15, 'data_volume': 0.5},

    'SMS': {'avg_duration': 0.0, 'frequency': 12, 'data_volume': 0.0},

    'Streaming': {'avg_duration': 0.0, 'frequency': 6, 'data_volume': 2.0}

}


# Generate network usage records

usage_data = []

base_date = datetime(2024, 1, 1)


# Create 10,000 subscribers with 20-100 usage events each

for subscriber_num in range(1, 10001):

    subscriber_id = f"SUB{subscriber_num:08d}"
    
    # Each subscriber gets 20-100 usage events over 12 months

    num_events = random.randint(20, 100)
    
    for i in range(num_events):

        # Spread usage events over 12 months

        days_offset = random.randint(0, 365)

        usage_date = base_date + timedelta(days=days_offset)
        
        # Add realistic timing (more usage during business hours and evenings)

        hour_weights = [1, 1, 1, 1, 1, 2, 4, 6, 8, 7, 6, 8, 9, 8, 7, 6, 8, 9, 10, 8, 6, 4, 3, 2]

        hours_offset = random.choices(range(24), weights=hour_weights)[0]

        usage_date = usage_date.replace(hour=hours_offset, minute=random.randint(0, 59), second=0, microsecond=0)
        
        # Select service type

        service_type = random.choice(SERVICE_TYPES)

        params = USAGE_PARAMS[service_type]
        
        # Calculate usage metrics with variability

        if service_type == 'Voice':

            duration_variation = random.uniform(0.3, 3.0)

            call_duration = round(params['avg_duration'] * duration_variation, 2)

            data_volume = 0.0

        elif service_type == 'Data':

            data_variation = random.uniform(0.1, 5.0)

            data_volume = round(params['data_volume'] * data_variation, 3)

            call_duration = 0.0

        elif service_type == 'SMS':

            data_volume = 0.0

            call_duration = 0.0

        else:  # Streaming

            data_variation = random.uniform(0.5, 8.0)

            data_volume = round(params['data_volume'] * data_variation, 3)

            call_duration = 0.0
        
        # Select cell tower and signal quality

        cell_tower_id = random.choice(CELL_TOWERS)

        # Signal quality varies by tower and time

        base_signal = random.randint(60, 95)

        signal_variation = random.randint(-15, 5)

        signal_quality = max(0, min(100, base_signal + signal_variation))
        
        usage_data.append({

            "subscriber_id": subscriber_id,

            "usage_date": usage_date,

            "service_type": service_type,

            "data_volume": data_volume,

            "call_duration": call_duration,

            "cell_tower_id": cell_tower_id,

            "signal_quality": signal_quality

        })



print(f"Generated {len(usage_data)} network usage records")

print("Sample record:", usage_data[0])

Generated 600441 network usage records
Sample record: {'subscriber_id': 'SUB00000001', 'usage_date': datetime.datetime(2024, 10, 11, 9, 9), 'service_type': 'Voice', 'data_volume': 0.0, 'call_duration': 6.73, 'cell_tower_id': 'TOWER_CHI_003', 'signal_quality': 65}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [1]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_usage = spark.createDataFrame(usage_data, schema=data_schema)


# Display schema and sample data

print("DataFrame Schema:")

df_usage.printSchema()



print("\nSample Data:")

df_usage.show(5)


# Insert data into Delta table with liquid clustering

# The TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (subscriber_id, usage_date) will automatically optimize the data layout

df_usage.write.mode("overwrite").insertInto("telecom.analytics.network_usage_uf")


print(f"\nSuccessfully inserted {df_usage.count()} records into telecom.analytics.network_usage_uf")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- subscriber_id: string (nullable = true)
 |-- usage_date: timestamp (nullable = true)
 |-- service_type: string (nullable = true)
 |-- data_volume: double (nullable = true)
 |-- call_duration: double (nullable = true)
 |-- cell_tower_id: string (nullable = true)
 |-- signal_quality: integer (nullable = true)


Sample Data:
+-------------+-------------------+------------+-----------+-------------+-------------+--------------+
|subscriber_id|         usage_date|service_type|data_volume|call_duration|cell_tower_id|signal_quality|
+-------------+-------------------+------------+-----------+-------------+-------------+--------------+
|  SUB00000001|2024-10-11 09:09:00|       Voice|        0.0|         6.73|TOWER_CHI_003|            65|
|  SUB00000001|2024-09-01 10:33:00|       Voice|        0.0|          7.6|TOWER_SEA_007|            64|
|  SUB00000001|2024-06-09 18:09:00|       Voice|        0.0|         5.82|TOWER_HOU_004|            91|
|  SUB00000001|2024-10-1


Successfully inserted 600441 records into telecom.analytics.network_usage_uf
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Subscriber usage history** (clustered by subscriber_id)
2. **Time-based network analysis** (clustered by usage_date)
3. **Combined subscriber + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [1]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Subscriber usage history - benefits from subscriber_id clustering

print("=== Query 1: Subscriber Usage History ===")

subscriber_history = spark.sql("""

SELECT subscriber_id, usage_date, service_type, data_volume, call_duration, signal_quality

FROM telecom.analytics.network_usage_uf

WHERE subscriber_id = 'SUB00000001'

ORDER BY usage_date DESC

""")



subscriber_history.show()

print(f"Records found: {subscriber_history.count()}")



# Query 2: Time-based network quality analysis - benefits from usage_date clustering

print("\n=== Query 2: Recent Network Quality Issues ===")

network_quality = spark.sql("""

SELECT usage_date, subscriber_id, cell_tower_id, signal_quality, service_type

FROM telecom.analytics.network_usage_uf

WHERE usage_date >= '2024-06-01' AND signal_quality < 50

ORDER BY signal_quality ASC, usage_date DESC

""")



network_quality.show()

print(f"Network quality issues found: {network_quality.count()}")



# Query 3: Combined subscriber + time query - optimal for our clustering strategy

print("\n=== Query 3: Subscriber Data Usage Trends ===")

usage_trends = spark.sql("""

SELECT subscriber_id, usage_date, service_type, data_volume, call_duration

FROM telecom.analytics.network_usage_uf

WHERE subscriber_id LIKE 'SUB000000%' AND usage_date >= '2024-04-01'

ORDER BY subscriber_id, usage_date

""")



usage_trends.show()

print(f"Usage trend records found: {usage_trends.count()}")

=== Query 1: Subscriber Usage History ===


+-------------+-------------------+------------+-----------+-------------+--------------+
|subscriber_id|         usage_date|service_type|data_volume|call_duration|signal_quality|
+-------------+-------------------+------------+-----------+-------------+--------------+
|  SUB00000001|2024-12-23 19:56:00|   Streaming|      9.203|         0.00|            64|
|  SUB00000001|2024-12-18 14:35:00|         SMS|      0.000|         0.00|            49|
|  SUB00000001|2024-12-13 07:58:00|         SMS|      0.000|         0.00|            64|
|  SUB00000001|2024-12-08 12:02:00|        Data|      0.587|         0.00|            57|
|  SUB00000001|2024-11-28 09:17:00|        Data|      1.336|         0.00|            80|
|  SUB00000001|2024-11-26 14:57:00|       Voice|      0.000|         1.91|            64|
|  SUB00000001|2024-11-15 20:26:00|       Voice|      0.000|         8.90|            68|
|  SUB00000001|2024-11-05 20:19:00|   Streaming|      6.681|         0.00|            65|
|  SUB0000

Records found: 65

=== Query 2: Recent Network Quality Issues ===


+-------------------+-------------+-------------+--------------+------------+
|         usage_date|subscriber_id|cell_tower_id|signal_quality|service_type|
+-------------------+-------------+-------------+--------------+------------+
|2024-12-31 23:30:00|  SUB00000489|TOWER_SFO_006|            45|   Streaming|
|2024-12-31 19:26:00|  SUB00002960|TOWER_LAX_002|            45|   Streaming|
|2024-12-31 13:51:00|  SUB00005928|TOWER_NYC_001|            45|   Streaming|
|2024-12-29 20:40:00|  SUB00003337|TOWER_HOU_004|            45|   Streaming|
|2024-12-28 13:04:00|  SUB00002745|TOWER_NYC_001|            45|   Streaming|
|2024-12-27 10:03:00|  SUB00007176|TOWER_NYC_001|            45|        Data|
|2024-12-26 19:25:00|  SUB00003629|TOWER_CHI_003|            45|         SMS|
|2024-12-26 08:18:00|  SUB00008098|TOWER_MIA_005|            45|        Data|
|2024-12-26 07:37:00|  SUB00006040|TOWER_SEA_007|            45|   Streaming|
|2024-12-26 01:45:00|  SUB00009227|TOWER_HOU_004|            45|

Network quality issues found: 7052

=== Query 3: Subscriber Data Usage Trends ===


+-------------+-------------------+------------+-----------+-------------+
|subscriber_id|         usage_date|service_type|data_volume|call_duration|
+-------------+-------------------+------------+-----------+-------------+
|  SUB00000001|2024-04-13 06:35:00|         SMS|      0.000|         0.00|
|  SUB00000001|2024-04-16 04:46:00|       Voice|      0.000|         5.62|
|  SUB00000001|2024-04-16 20:47:00|         SMS|      0.000|         0.00|
|  SUB00000001|2024-04-24 22:28:00|       Voice|      0.000|         4.94|
|  SUB00000001|2024-05-03 18:50:00|        Data|      1.165|         0.00|
|  SUB00000001|2024-05-05 02:55:00|   Streaming|     10.595|         0.00|
|  SUB00000001|2024-05-12 06:30:00|       Voice|      0.000|        11.42|
|  SUB00000001|2024-05-23 09:40:00|         SMS|      0.000|         0.00|
|  SUB00000001|2024-05-23 19:20:00|         SMS|      0.000|         0.00|
|  SUB00000001|2024-05-29 23:50:00|        Data|      1.616|         0.00|
|  SUB00000001|2024-06-07

Usage trend records found: 4695


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the telecommunications insights possible with this optimized structure.

### Key Analytics

- **Subscriber usage patterns** and data consumption analysis
- **Network performance metrics** and signal quality trends
- **Service type adoption** and usage distribution
- **Cell tower utilization** and capacity planning

In [1]:
# Analyze clustering effectiveness and telecommunications insights


# Subscriber usage analysis

print("=== Subscriber Usage Analysis ===")

subscriber_usage = spark.sql("""

SELECT subscriber_id, COUNT(*) as total_sessions,

       ROUND(SUM(data_volume), 3) as total_data_gb,

       ROUND(SUM(call_duration), 2) as total_call_minutes,

       ROUND(AVG(signal_quality), 2) as avg_signal_quality,

       COUNT(DISTINCT service_type) as services_used

FROM telecom.analytics.network_usage_uf

GROUP BY subscriber_id

ORDER BY total_data_gb DESC

""")



subscriber_usage.show()


# Service type usage patterns

print("\n=== Service Type Usage Patterns ===")

service_patterns = spark.sql("""

SELECT service_type, COUNT(*) as total_usage,

       ROUND(SUM(data_volume), 3) as total_data_gb,

       ROUND(SUM(call_duration), 2) as total_call_minutes,

       ROUND(AVG(signal_quality), 2) as avg_signal_quality,

       COUNT(DISTINCT subscriber_id) as unique_subscribers

FROM telecom.analytics.network_usage_uf

GROUP BY service_type

ORDER BY total_usage DESC

""")



service_patterns.show()


# Cell tower performance

print("\n=== Cell Tower Performance ===")

tower_performance = spark.sql("""

SELECT cell_tower_id, COUNT(*) as total_connections,

       COUNT(DISTINCT subscriber_id) as unique_subscribers,

       ROUND(AVG(signal_quality), 2) as avg_signal_quality,

       ROUND(SUM(data_volume), 3) as total_data_gb,

       ROUND(SUM(call_duration), 2) as total_call_minutes

FROM telecom.analytics.network_usage_uf

GROUP BY cell_tower_id

ORDER BY total_connections DESC

""")



tower_performance.show()


# Hourly usage patterns

print("\n=== Hourly Usage Patterns ===")

hourly_patterns = spark.sql("""

SELECT HOUR(usage_date) as hour_of_day, COUNT(*) as usage_events,

       ROUND(SUM(data_volume), 3) as data_volume_gb,

       ROUND(SUM(call_duration), 2) as call_minutes,

       ROUND(AVG(signal_quality), 2) as avg_signal_quality

FROM telecom.analytics.network_usage_uf

GROUP BY HOUR(usage_date)

ORDER BY hour_of_day

""")



hourly_patterns.show()


# Monthly network trends

print("\n=== Monthly Network Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(usage_date, 'yyyy-MM') as month,

       COUNT(*) as total_usage,

       ROUND(SUM(data_volume), 3) as monthly_data_gb,

       ROUND(SUM(call_duration), 2) as monthly_call_minutes,

       ROUND(AVG(signal_quality), 2) as avg_signal_quality,

       COUNT(DISTINCT subscriber_id) as active_subscribers

FROM telecom.analytics.network_usage_uf

GROUP BY DATE_FORMAT(usage_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Subscriber Usage Analysis ===


+-------------+--------------+-------------+------------------+------------------+-------------+
|subscriber_id|total_sessions|total_data_gb|total_call_minutes|avg_signal_quality|services_used|
+-------------+--------------+-------------+------------------+------------------+-------------+
|  SUB00007721|           100|      437.600|            186.18|             71.52|            4|
|  SUB00000310|            87|      360.635|            115.96|             74.31|            4|
|  SUB00009487|            99|      359.354|            142.81|             71.95|            4|
|  SUB00004489|           100|      355.142|            241.89|             72.93|            4|
|  SUB00009757|            99|      353.809|            159.57|              72.8|            4|
|  SUB00003440|           100|      352.415|            206.11|             73.61|            4|
|  SUB00001442|            97|      349.590|            165.55|             72.38|            4|
|  SUB00007546|            96|

+------------+-----------+-------------+------------------+------------------+------------------+
|service_type|total_usage|total_data_gb|total_call_minutes|avg_signal_quality|unique_subscribers|
+------------+-----------+-------------+------------------+------------------+------------------+
|   Streaming|     150681|  1281179.011|              0.00|             72.53|              9998|
|         SMS|     150362|        0.000|              0.00|             72.54|              9999|
|        Data|     149979|   191224.751|              0.00|             72.46|              9998|
|       Voice|     149419|        0.000|        1233854.92|             72.49|              9998|
+------------+-----------+-------------+------------------+------------------+------------------+


=== Cell Tower Performance ===


+-------------+-----------------+------------------+------------------+-------------+------------------+
|cell_tower_id|total_connections|unique_subscribers|avg_signal_quality|total_data_gb|total_call_minutes|
+-------------+-----------------+------------------+------------------+-------------+------------------+
|TOWER_SEA_007|            86303|              9961|              72.5|   211831.705|         178234.77|
|TOWER_LAX_002|            85815|              9972|             72.54|   210171.637|         175583.76|
|TOWER_MIA_005|            85807|              9963|             72.46|   209335.766|         176913.40|
|TOWER_HOU_004|            85665|              9967|             72.49|   211980.345|         175157.25|
|TOWER_SFO_006|            85652|              9951|             72.49|   209578.515|         176548.76|
|TOWER_NYC_001|            85627|              9952|              72.5|   209938.392|         175181.60|
|TOWER_CHI_003|            85572|              9947|   

+-----------+------------+--------------+------------+------------------+
|hour_of_day|usage_events|data_volume_gb|call_minutes|avg_signal_quality|
+-----------+------------+--------------+------------+------------------+
|          0|        4882|     11751.071|     9743.80|             72.72|
|          1|        4784|     11658.243|    10049.24|              72.4|
|          2|        4700|     11674.221|     9564.95|             72.35|
|          3|        4600|     11147.394|     9624.37|             72.65|
|          4|        4825|     12138.186|     9876.13|              72.7|
|          5|        9596|     23752.194|    19921.37|             72.28|
|          6|       19040|     47192.768|    39631.71|             72.37|
|          7|       28752|     70402.197|    58563.54|             72.52|
|          8|       37945|     93032.801|    78315.74|             72.48|
|          9|       33081|     81710.376|    66832.08|             72.48|
|         10|       28429|     68958.8

+-------+-----------+---------------+--------------------+------------------+------------------+
|  month|total_usage|monthly_data_gb|monthly_call_minutes|avg_signal_quality|active_subscribers|
+-------+-----------+---------------+--------------------+------------------+------------------+
|2024-01|      50546|     124398.149|           103286.56|             72.46|              9742|
|2024-02|      47868|     116160.897|            97495.47|             72.52|              9681|
|2024-03|      51080|     124101.472|           105340.74|             72.51|              9767|
|2024-04|      49091|     120063.291|           102222.16|             72.51|              9708|
|2024-05|      51182|     125372.721|           105244.69|             72.44|              9739|
|2024-06|      49228|     121975.716|           100510.07|             72.45|              9710|
|2024-07|      50892|     125537.807|           104053.12|             72.57|              9779|
|2024-08|      50955|     1250

## Key Takeaways: Iceberg and Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `TBLPROPERTIES('delta.universalFormat.enabledFormats' = 'iceberg') CLUSTER BY (subscriber_id, usage_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (subscriber_id, usage_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Telecommunications analytics where network monitoring and customer experience are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for telecommunications data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles telecommunications-scale data volumes effortlessly

### Best Practices for Iceberg and Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger telecommunications datasets
- Integrate with real network monitoring systems and CDR data

This notebook demonstrates how Oracle AI Data Platform combines Delta's advanced liquid clustering with Iceberg's open, future-proof architecture to deliver enterprise-grade analytics that are both high-performance and standards-compliant.